In [86]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col



In [87]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "C:\\MongoDBSparkCOnnector\\mongo-spark-connector_2.12-3.0.2.jar,C:\\MongoDBSparkConnector\\mongo-java-driver-3.12.9.jar" pyspark-shell'

In [88]:
spark = SparkSession.builder \
    .appName("MongoDB and PySpark") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/GoT.twitter") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/GoT.twitter") \
    .getOrCreate()

In [89]:

df = spark.read.csv('C:\\data files\\June2016 GOT.csv',inferSchema=True, header =True)


In [90]:
df.printSchema()

root
 |-- poster: string (nullable = true)
 |-- recipient: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- tweet date: string (nullable = true)
 |-- tweet id: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- retweet status: string (nullable = true)



In [91]:
df.show()

+--------------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|              poster|      recipient|relationship|          tweet date|          tweet id|               tweet|            hashtags|retweet status|
+--------------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|               M04th|         Cutevl|       reply|Wed Jun 01 06:32:...|737894675277852674|@Cutevl انا اصدار...|                  []|         False|
|          aRnecutska|chris_voloshina|    mentions|Wed Jun 01 06:34:...|737895014991302656|RT @chris_voloshi...|                  []|          True|
|          aRnecutska|     aRnecutska|    mentions|Wed Jun 01 06:34:...|737895014991302656|RT @chris_voloshi...|                  []|          True|
|        Macready1306|   Macready1306|       tweet|Wed Jun 01 06:35:...|737895438624423937|I've never watc

In [92]:
#Get rid of false positives
#Drop all where hashtag != gameofthrones

df = df.withColumn("hashtags", lower(col("hashtags")))
df = df.filter(df.hashtags.contains("gameofthrones"))


In [93]:
df.show()

+---------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|         poster|      recipient|relationship|          tweet date|          tweet id|               tweet|            hashtags|retweet status|
+---------------+---------------+------------+--------------------+------------------+--------------------+--------------------+--------------+
|   Macready1306|   Macready1306|       tweet|Wed Jun 01 06:35:...|737895438624423937|I've never watche...|   ['gameofthrones']|         False|
|  wahanapenulis|  wahanapenulis|       tweet|Wed Jun 01 06:38:...|737896168445870080|I'm Tyrion. Disco...|   ['gameofthrones']|         False|
|  scopece_marco|EmiliaClarkeNet|    mentions|Wed Jun 01 06:44:...|737897535788961792|RT @EmiliaClarkeN...|   ['gameofthrones']|          True|
|  scopece_marco|   emiliaclarke|    mentions|Wed Jun 01 06:44:...|737897535788961792|RT @EmiliaClarkeN...|   ['gameofthrones']|        

In [94]:
#Store in MongoDB
df.write.format("mongo").mode("append").save()



In [95]:
 spark.stop()